<a href="https://colab.research.google.com/github/aiscience-22/UA_War/blob/twitter_data_cleanup_0.04/Twitter_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-22 03:43:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.23MB/s    in 0.2s    

2022-08-22 03:43:06 (6.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# Add drivers to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://uaresources.s3.amazonaws.com/aug_twitter_sentiments.csv"

spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("aug_twitter_sentiments.csv"), sep=",", header=True, inferSchema=True)


# Show DataFrame
user_data_df.show()

+-------------------+--------------------+--------------------+-------------------+--------------------+
|     tweetcreatedts|        cleaned_text|            negative|            neutral|            positive|
+-------------------+--------------------+--------------------+-------------------+--------------------+
|2022-08-01 00:00:00|the conflict is b...|  0.6027116775512695|0.38163381814956665|  0.0156544279307127|
|2022-08-01 00:00:00|remember when sma...| 0.21225015819072723| 0.7508949041366577|0.036854907870292664|
|2022-08-01 00:00:01|hi  you have to u...|  0.4901295006275177|   0.46605184674263|0.043818794190883636|
|2022-08-01 00:00:01|the world is in d...|  0.8760228157043457|0.11690670996904373|0.007070485968142748|
|2022-08-01 00:00:01|will the imposed ...| 0.04450933262705803| 0.9108507633209229| 0.04463988170027733|
|2022-08-01 00:00:01|owner of ukraines...|  0.8333820104598999| 0.1629866510629654|  0.0036314248573035|
|2022-08-01 00:00:01|animal shelter do...|0.01331210788

In [10]:
# Read in data from S3 Buckets
url ="https://uaresources.s3.amazonaws.com/EventsUA.csv"
spark.sparkContext.addFile(url)
event_df = spark.read.csv(SparkFiles.get("EventsUA.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
event_df.show()

+---+--------+--------------------+------+
|_c0|    Date|               Event|Weight|
+---+--------+--------------------+------+
|  1| 2/24/22|Start Russian Inv...|   1.0|
|  2|2/25/202|Battles for Kyiv,...|  0.75|
|  3|  3/1/22|Bombing of Babiy ...|  0.75|
|  4|  3/3/22|Russian attack fo...|  0.75|
|  5|  3/6/22|Bombing of the th...|  0.75|
|  6|  3/9/22|Bombing of the ma...|  0.75|
|  7| 3/13/22|Documentary maker...|   0.3|
|  8| 3/24/22|Surviving prisone...|   0.3|
|  9|  4/3/22|Evidence of massa...|  0.75|
| 10|  4/8/22|The shelling of t...|  0.75|
| 11|  4/9/22|British Prime Min...|   0.3|
| 12| 4/14/22|Sinking of the Ru...|   0.5|
| 13| 4/28/22|The U.S. House of...|   0.5|
| 14| 5/16/22|Exit of the wound...|   0.3|
| 15| 6/27/22|Rocket attack on ...|   0.5|
| 16| 7/28/22|Torture and murde...|   0.3|
+---+--------+--------------------+------+



In [12]:
# Join the two DataFrame
joined_df=user_data_df.join(event_df, on='Date', how='left')
joined_df.show()

AnalysisException: ignored

In [ ]:
# Drop null values
dropna_df=joined_df.dropna()
dropna_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [ ]:
# Filter for active users
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df=dropna_df.filter(col('active_user') ==True)
cleaned_df.show()

+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|     username| id|first_name|  last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|   fstappard5|  6|    Fraser|    Korneev|       true|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|   lhambling6|  7|    Demott|     Rapson|       true|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|    wheinerte| 15|   Sadella|      Jaram|       true|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
| droughsedgeg| 17|    Hewitt|    Trammel|       true|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|    ydudeniei| 19|       Ted|    Knowlys|       true|      31 South Drive|                Ohio| 

In [ ]:
# Create user DataFrame to match active_user table
clean_user_df=cleaned_df.select(['id', 'first_name', 'last_name', 'username'])
clean_user_df.show()

+---+----------+-----------+-------------+
| id|first_name|  last_name|     username|
+---+----------+-----------+-------------+
|  6|    Fraser|    Korneev|   fstappard5|
|  7|    Demott|     Rapson|   lhambling6|
| 15|   Sadella|      Jaram|    wheinerte|
| 17|    Hewitt|    Trammel| droughsedgeg|
| 19|       Ted|    Knowlys|    ydudeniei|
| 23|  Annmarie|     Lafond|     fmyttonm|
| 28|      Toma|     Sokell|   bfletcherr|
| 30|       Ram|    Lefever|     gturleyt|
| 31|    Raddie|    Heindle|    calyukinu|
| 33|    Wallie|       Caws| ckleinlererw|
| 34|    Derril|Varfolomeev|  pshanklandx|
| 39|     Kelcy|     Wheway|    enelane12|
| 40|    Dorree|    Rookeby|    sfollet13|
| 41|    Martyn|       Tott|      mtesh14|
| 43|     Cally|      Thody|   tseyfart16|
| 45|       Ted|   Pittaway|   hfarrier18|
| 48|      Fifi|    Lidgley|     nabbie1b|
| 50|    Ashely|     O'Hern|  ystadding1d|
| 53|   Diannne|Osbaldeston|hhallgalley1g|
| 60|     Sonny|     Jeskin|   ageaveny1n|
+---+------

In [ ]:
# Create a DataFrame to match the billing_info table
clean_billing_df=cleaned_df.select(['billing_id', 'street_address', 'state', 'username'])
clean_billing_df.show()

+----------+--------------------+--------------------+-------------+
|billing_id|      street_address|               state|     username|
+----------+--------------------+--------------------+-------------+
|         6|  76084 Novick Court|           Minnesota|   fstappard5|
|         7|    86320 Dahle Park|District of Columbia|   lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|    wheinerte|
|        17|    2455 Corry Alley|      North Carolina| droughsedgeg|
|        19|      31 South Drive|                Ohio|    ydudeniei|
|        23|     35 Oriole Place|             Georgia|     fmyttonm|
|        28|39641 Eggendart Hill|            Maryland|   bfletcherr|
|        30|   9969 Laurel Alley|               Texas|     gturleyt|
|        31|   811 Talmadge Road|                Ohio|    calyukinu|
|        33|   9999 Kenwood Pass|              Oregon| ckleinlererw|
|        34|     4 Jenifer Court|             Florida|  pshanklandx|
|        39|93207 Morningstar...| 

In [ ]:
# Create a DataFrame to match the payment_info table
clean_payment_df=cleaned_df.select(['billing_id', 'cc_encrypted'])
clean_payment_df.show()

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
|        23|a799fcafe47d7fb19...|
|        28|a799fcafe47d7fb19...|
|        30|a799fcafe47d7fb19...|
|        31|a799fcafe47d7fb19...|
|        33|a799fcafe47d7fb19...|
|        34|a799fcafe47d7fb19...|
|        39|a799fcafe47d7fb19...|
|        40|a799fcafe47d7fb19...|
|        41|a799fcafe47d7fb19...|
|        43|a799fcafe47d7fb19...|
|        45|a799fcafe47d7fb19...|
|        48|a799fcafe47d7fb19...|
|        50|a799fcafe47d7fb19...|
|        53|a799fcafe47d7fb19...|
|        60|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 20 rows



**Get our transformed raw data into our database**

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password:')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://databiz.c3x2frc4y9xs.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password:··········


In [ ]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)